In [1]:
import importlib
import sys
import torch
from torch.utils.data import random_split
import random
from collections.abc import Iterator
from tqdm.notebook import tqdm
from Levenshtein import distance
import collections
import pickle
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import CRPS.CRPS as pscore
import os


sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')


#from model.dropout_uncertainty_enc_dec_LSTM_new_dec.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM
from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [4]:
#load model
file_path_model = '../../../training_variational_dropout/Helpdesk/Helpdesk_full_no_grad_norm_new_2.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)



# Load the dataset
file_path_data_set = '../../../../../encoded_data/helpdesk_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('Activity', 16, {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'EOS': 5, 'INVALID': 6, 'Insert ticket': 7, 'RESOLVED': 8, 'Require upgrade': 9, 'Resolve SW anomaly': 10, 'Resolve ticket': 11, 'Schedule intervention': 12, 'Take in charge ticket': 13, 'VERIFIED': 14, 'Wait': 15}), ('Resource', 24, {'EOS': 1, 'Value 1': 2, 'Value 10': 3, 'Value 11': 4, 'Value 12': 5, 'Value 13': 6, 'Value 14': 7, 'Value 15': 8, 'Value 16': 9, 'Value 17': 10, 'Value 18': 11, 'Value 19': 12, 'Value 2': 13, 'Value 20': 14, 'Value 21': 15, 'Value 22': 16, 'Value 3': 17, 'Value 4': 18, 'Value 5': 19, 'Value 6': 20, 'Value 7': 21, 'Value 8': 22, 'Value 9': 23}), ('Variant index', 166, {'1.0': 1, '10.0': 2, '100.0': 3, '101.0': 4, '102.0': 5, '103.0': 6, '104.0': 7, '105.0': 8, '106.0': 9, '107.0': 10, '108.0': 11, '109.0': 12, '11.0': 13, '110.0': 14, '111.0': 15, '112.0': 16, '113.0': 17, '114.0': 18, '12.0': 19, '13.0': 20, '14.0': 21, '15.0': 22, '16.0

In [5]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   concept_name='Activity',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   )


In [6]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")


output_dir = '../../../../../evaluation_results/Helpdesk/no_gn'
save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=True)):
    print(case_name)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/916 [00:00<?, ?it/s]

Case 1863
1 3
Case 1863
2 2
Case 1863
3 1
Case 2003
1 7
Case 1814
1 4
Case 2003
2 6
Case 1814
2 3
Case 2003
3 5
Case 1814
3 2
Case 1814
4 1
Case 2003
4 4
Case 2003
5 3
Case 2045
1 3
Case 2003
6 2
Case 2003
7 1
Case 2045
2 2
Case 2158
1 3
Case 2045
3 1
Case 1732
1 3
Case 2158
2 2
Case 2158
3 1
Case 1732
2 2
Case 2155
1 3
Case 2195
1 3
Case 1732
3 1
Case 2155
2 2
Case 2155
3 1
Case 2195
2 2
Case 2195
3 1
Case 2057
1 3
Case 2057
2 2
Case 2315
1 3
Case 2057
3 1
Case 2315
2 2
Case 2315
3 1
Case 2305
1 3
Case 2305
2 2
Case 2305
3 1
Case 2240
1 3
Case 2240
2 2
Case 2240
3 1
Case 2007
1 5
Case 2007
2 4
Case 2007
3 3
Case 2007
4 2
Case 2007
5 1
Case 1833
1 3
Case 1833
3 1
Case 1833
2 2
Case 2342
1 5
Saved 50 results to ../../../../../evaluation_results/Helpdesk/no_gn/results_part_050.pkl
Case 2342
2 4
Case 2342
3 3
Case 2342
4 2
Case 2342
5 1
Case 1619
2 2
Case 1619
1 3
Case 1619
3 1
Case 2176
1 3
Case 2176
2 2
Case 2176
3 1
Case 2257
1 3
Case 2257
2 2
Case 2257
3 1
Case 1778
1 2
Case 1778
2 1


: 